In [22]:
from bs4 import BeautifulSoup
import requests
from lxml import html
from lxml.cssselect import CSSSelector
import re

In [139]:
url = 'https://www.finn.no/realestate/homes/search.html?location=0.20061&page=2'

In [145]:
response = requests.get(url)

In [146]:
soup = BeautifulSoup(response.content, 'html.parser')

In [147]:
links = []
for tag in page_results:
    res = tag.find_all('a')
    try:
        links.append(res[0].get('href'))
    except IndexError:
        pass
links = ['https://finn.no'+l for l in links if 'finnkode' in l]

In [246]:
url = links[1]

In [262]:
# TMP
url = links[16]
print(url)
page = requests.get(url)
tree = html.fromstring(page.content)
for i in range(11):
    print('')
    print(tree.xpath(f'//section[@class="panel"]//dt[{i}]/text()'))
    print(tree.xpath(f'//section[@class="panel"]//dd[{i}]/text()'))

https://finn.no/realestate/homes/ad.html?finnkode=167025183

[]
[]

['Boligtype', 'Tomteareal', 'Fellesformue']
['Leilighet', '33931 m²', '28\xa0999 kr']

['Eieform', 'Bruttoareal']
['Andel', '73 m²']

['Soverom']
['3']

['Primærrom']
['66 m²']

['Bruksareal']
['66 m²']

['Etasje']
['3']

['Byggeår']
['1962']

['Rom']
['4']

[]
[]

[]
[]


In [215]:
#url = 'https://www.finn.no/realestate/homes/ad.html?finnkode=171155350'

In [230]:
page = requests.get(url)
tree = html.fromstring(page.content)

['Boligtype', 'Rom', 'Formuesverdi', 'Mobil']
['Leilighet', '7', '1\xa0286\xa0425 kr', '\n                    ', '\n                        ']


In [218]:
tree.xpath('//section[@class="panel"]//p[@class="u-caption"]/text()')[0]

'Trondheimsveien 8, 0560 Oslo'

In [219]:
extract_digits(tree.xpath('//div[@class="panel"]//span[@class="u-t3"]/text()')[0])

'6600000'

In [220]:
d = {}
for i, _ in enumerate(tree.xpath('//div[@class="panel"]//dl[@class="definition-list"]/dt')):
    field = tree.xpath(f'//div[@class="panel"]//dl[@class="definition-list"]/dt[{i+1}]/text()')[0]
    value = extract_digits(tree.xpath(f'//div[@class="panel"]//dl[@class="definition-list"]/dd[{i+1}]/text()')[0])
    print(field)
    print(value)

Fellesgjeld
76000
Omkostninger
168242
Totalpris
6844242
Felleskost/mnd.
3687


In [221]:
d = {}
for i, _ in enumerate(tree.xpath('//div[@class="panel"]//dl[@class="definition-list"]/dt')):
    field = tree.xpath(f'//div[@class="panel"]//dl[@class="definition-list"]/dt[{i+1}]/text()')[0]
    value = extract_digits(tree.xpath(f'//div[@class="panel"]//dl[@class="definition-list"]/dd[{i+1}]/text()')[0])
    if field == 'Omkostninger':
        d['fees'] = value
    elif field == 'Totalpris':
        d['total_price'] = value
    elif field == 'Felleskost/mnd.':
        d['overheads'] = value
    elif field == 'Fellesgjeld':
        d['joint_debt'] = value
d

{'joint_debt': '76000',
 'fees': '168242',
 'total_price': '6844242',
 'overheads': '3687'}

In [278]:
tree.xpath(f'//section[@class="panel"]//dt[{1}]/text()')

['Boligtype', 'Tomteareal', 'Fellesformue']

In [265]:
selected_fields = ['Boligtype', 'Eieform', 'Soverom', 'Primærrom', 'Bruksareal', 'Etasje',
                  'Byggeår', 'Energimerking', 'Rom', 'Tomteareal', 'Bruttoareal', 'Fellesformue', 'Formuesverdi']

In [284]:
fields = []
values = []
final_fields = []
final_values = []
for i, _ in enumerate(tree.xpath('//section[@class="panel"]//dt')):
    fields = tree.xpath(f'//section[@class="panel"]//dt[{i+1}]/text()')
    values = tree.xpath(f'//section[@class="panel"]//dd[{i+1}]/text()')
    for field in fields:
        if field in selected_fields:
            final_fields.append(field)
            final_values.append(values[fields.index(field)])

In [285]:
final_fields

['Boligtype',
 'Tomteareal',
 'Fellesformue',
 'Eieform',
 'Bruttoareal',
 'Soverom',
 'Primærrom',
 'Bruksareal',
 'Etasje',
 'Byggeår',
 'Rom']

In [287]:
for field, value in zip(final_fields, final_values):
    print(field, value)
    d = extract_field_value_pairs(field, value, d)

Boligtype Leilighet
Tomteareal 33931 m²
Fellesformue 28 999 kr
Eieform Andel
Bruttoareal 73 m²
Soverom 3
Primærrom 66 m²
Bruksareal 66 m²
Etasje 3
Byggeår 1962
Rom 4


In [288]:
d

{'joint_debt': '76000',
 'fees': '168242',
 'total_price': '6844242',
 'overheads': '3687',
 'type': 'Leilighet',
 'lot_space': '33931',
 'joint_capital': '28999',
 'tenure': 'Andel',
 'gross_floor_space': '73',
 'bedrooms': '3',
 'net_internal_area': '66',
 'gross_internal_area': '66',
 'floor': '3',
 'year_of_construction': '1962',
 'rooms': '4'}

In [228]:
i = 7
print(tree.xpath(f'//section[@class="panel"]//dt[{i}]/text()'))
print(tree.xpath(f'//section[@class="panel"]//dd[{i}]/text()'))

['Byggeår']
['1892']


['66 m²']

In [207]:
d

{'joint_debt': '367000',
 'fees': '1132',
 'total_price': '2858132',
 'overheads': '5489',
 'type': 'Leilighet',
 'rooms': '3',
 'asset_value': '992\xa0498 kr',
 'tenure': '\n                    ',
 'lot_space': '',
 'bedrooms': 'Andel',
 'gross_floor_space': '27418',
 'net_internal_area': '2',
 'gross_internal_area': '77',
 'floor': '66 m²',
 'year_of_construction': '70 m²',
 'energy_class': ''}

In [137]:
d['last_edited'] = tree.xpath('//section[@aria-labelledby="ad-info-heading"]//tr/td/text()')[0]

In [138]:
d

{'fees': '389902',
 'total_price': '15289902',
 'overheads': '5100',
 'last_edited': '25. feb 2020 17:32'}

In [126]:
def extract_digits(string):
    return re.sub('\D', '', string)

def extract_field_value_pairs(field: str, value: str, d: dict) -> dict:
    # What is usually in 'key info'
    if field == 'Boligtype':
        d['type'] = value
    elif field == 'Eieform':
        d['tenure'] = value
    elif field == 'Soverom':
        d['bedrooms'] = value
    elif field == 'Primærrom':
        d['net_internal_area'] = extract_digits(value)
    elif field == 'Bruksareal':
        d['gross_internal_area'] = extract_digits(value)
    elif field == 'Etasje':
        d['floor'] = value
    elif field == 'Byggeår':
        d['year_of_construction'] = value
    elif field == 'Energimerking':
        d['energy_class'] = '' # TODO 
    # What is usually in 'more key info', but could come here
    if field == 'Rom':
        d['rooms'] = value
    elif field == 'Tomteareal':
        d['lot_space'] = extract_digits(value)
    elif field == 'Bruttoareal':
        d['gross_floor_space'] = extract_digits(value)
    elif field == 'Fellesformue':
        d['joint_capital'] = extract_digits(value)
    elif field == 'Formuesverdi':
        d['asset_value'] = extract_digits(value)

    return d